# Computer Vision III: Detection, Segmentation and Tracking (CV3DST) GNN Exercise

In this exercise, we will first develop an extension of the ReID-based tracker we created in the previous exercise that will make it more robust to occlusions by allowing it to recover from missed detections.

We will then implement a Message Passing Network from scratch, and we will use to build a model that will learn to combine position information and reid features to directly predict associations between past tracks and detections. We will use this model to create robust tracker.

Your tasks are the following:
- Adapt the track management scheme of our ReIDTracker allow it to recover from missed detections.
- Implement a Message Passing Network from scratch to operate on bipartite graphs
- Implement the pairwise feature  computation to obtain features for our Message Passing Network
- Train the Message Passing Network and improve your tracker's IDF1 score


## Setup

### Download and extract project data to your Google Drive

1.   **Required**: Please follow all instructions of exercise 0 before running this notebook.
2.   Save this notebook to your Google Drive by clicking `Save a copy in Drive` from the `File` menu.
3.   Download [this](https://vision.in.tum.de/webshare/u/cetintas/cv3dst_gnn_exercise.zip) zip file to your desktop, extract it and upload it into the `Colab Notebooks` folder in your Google Drive.

#### Connect the notebook to your Google Drive

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
root_dir = "gdrive/My Drive/Colab Notebooks/cv3dst_exercise/"
gnn_root_dir = "gdrive/My Drive/Colab Notebooks/cv3dst_gnn_exercise/"

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

!pip install tqdm lap
!pip install https://github.com/timmeinhardt/py-motmetrics/archive/fix_pandas_deprecating_warnings.zip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.5 MB 8.0 MB/s 
  Created wheel for lap: filename=lap-0.4.0-cp37-cp37m-linux_x86_64.whl size=1590229 sha256=2888d0d67e9d4f788ed6ac4ca631c069427678816452fc21851ea966b935ce5f
  Stored in directory: /root/.cache/pip/wheels/b1/0b/e3/ef9daf1b5547b56389e42c80c3100f1e6479bf5fd00fd9d6ba
Successfully built lap
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     \ 148 kB 4.7 MB/s
  Created wheel for motmetrics: filename=motmetrics-1.1.3-py3-none-any.whl size=134199 sha256=74da72b5f4f8e5004bb504428cddc2683d39b34d321f5d947f669b7b5e861953
  Stored in directory: /tmp/pip-ephem-wheel-cache-fby2lsaz/wheels/39/60/bf/90b1b02ff42db1bf7f2d2fa3eef2fe8bc46061182cf4ce7b37
Successfully built motmetrics


The `root_dir` path points to the directory and the content in your Google Drive.

#### Install and import Python libraries

In [ ]:
import os
import sys
sys.path.append(os.path.join(gnn_root_dir, 'src'))


import matplotlib.pyplot as plt
import numpy as np
import time
from tqdm.autonotebook import tqdm

import torch
from torch.utils.data import DataLoader

from tracker.data_track import MOT16Sequences
from tracker.tracker import Tracker, ReIDTracker
from tracker.utils import run_tracker, cosine_distance
from scipy.optimize import linear_sum_assignment as linear_assignment
import os.path as osp

import motmetrics as mm
mm.lap.default_solver = 'lap'

import sys
import os
sys.path.append(os.path.join(gnn_root_dir, 'src2'))
from tracker.object_detector import FRCNN_FPN
import torch
from PIL import Image


def crop_imgs(img, boxes, transforms):
    assert isinstance(img, Image.Image)

    crops = []
    for box in boxes:
        box_crop = img.crop(box)
        crops.append(transforms(box_crop))
    #return np.stack(crops)
    return torch.stack(crops)


def load_obj_detect(obj_detect_model_file):
    obj_detect_nms_thresh = 0.3

    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    # object detector
    obj_detect = FRCNN_FPN(num_classes=2, nms_thresh=obj_detect_nms_thresh)
    obj_detect_state_dict = torch.load(obj_detect_model_file,
                                    map_location=lambda storage, loc: storage)
    obj_detect.load_state_dict(obj_detect_state_dict)
    obj_detect.eval()
    obj_detect.cuda()

    return obj_detect

In [ ]:
! wget https://vision.in.tum.de/webshare/u/brasoand/cv3dst/resnet50_reid.pth

--2022-08-01 16:31:15--  https://vision.in.tum.de/webshare/u/brasoand/cv3dst/resnet50_reid.pth
Resolving vision.in.tum.de (vision.in.tum.de)... 131.159.19.110, 2a09:80c0:18::1110
Connecting to vision.in.tum.de (vision.in.tum.de)|131.159.19.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 86824473 (83M)
Saving to: ‘resnet50_reid.pth’

resnet50_reid.pth   100%[===================>]  82.80M  17.0MB/s    in 5.9s    

2022-08-01 16:31:22 (14.0 MB/s) - ‘resnet50_reid.pth’ saved [86824473/86824473]



In [ ]:
from torchvision import transforms as T
from market.models import build_model

obj_detect= load_obj_detect(os.path.join(root_dir, 'models', 'faster_rcnn_fpn.model'))
crop_size = (256, 128)
tforms = T.Compose((T.Resize(crop_size), T.ToTensor(), T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])))
reid_model = build_model('resnet34', 751, loss='softmax', pretrained=True)


reid_ckpt = torch.load('resnet50_reid.pth')
reid_model.load_state_dict(reid_ckpt)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:136: UserWarning: Using 'backbone_name' and 'weights' as positional parameter(s) is deprecated since 0.13 and will be removed in 0.15. Please use keyword parameter(s) instead.
  f"Using {sequence_to_str(tuple(keyword_only_kwargs.keys()), separate_last='and ')} as positional "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth


  0%|          | 0.00/83.3M [00:00<?, ?B/s]

<All keys matched successfully>

In [ ]:
!ls "gdrive/My Drive/Colab Notebooks/cv3dst_exercise/data/MOT16/train"
!ls "gdrive/My Drive/Colab Notebooks/cv3dst_exercise/data/MOT16/test"

MOT16-02  MOT16-04  MOT16-05  MOT16-09	MOT16-10  MOT16-11  MOT16-13
MOT16-01  MOT16-03  MOT16-06  MOT16-07	MOT16-08  MOT16-12  MOT16-14


In [ ]:
!ls 'gdrive/My Drive/Colab Notebooks/cv3dst_gnn_exercise/src'

gnn  market  tracker


## Speed-Ups
In order to speed up training and inference runtimes, in this exercise we will be working with pre-computed detections and ReID embeddings. We ran the object detector we provided in Exercise 0 and applied to all frames. We also computed reid embeddings for all boxes in every frame of the dataset so that they don't need to be computed every time you run your tracker. This yields over 10x speed improvements. You will not have to work directly with the resulting files, as we have internally adapted the boilerplate code to work with them.

In [ ]:
from tracker.data_track import MOT16Sequences
import os.path as osp
from torch.utils.data import DataLoader
from tqdm.autonotebook import tqdm

stored_data_filename = 'preprocessed_data_{}_.pth'

reid_model.eval().cuda()
obj_detect.eval().cuda()

for train_test in ('train', 'test'):
    db = {}
    sequences = MOT16Sequences(f'MOT16-{train_test}', osp.join(root_dir,'data/MOT16'), vis_threshold=0.)

    for seq in sequences:
        print(f"Processing sequence {seq}")
        data_loader = DataLoader(seq, batch_size=1, shuffle=False)
        db[str(seq)]= []
        for frame_num, frame in tqdm(enumerate(data_loader)):
            with torch.no_grad():
                img_path = frame['img_path'][0]
                img = Image.open(img_path)

                assert frame_num + 1 == int(img_path.split('/')[-1].split('.')[0])

                # Store detected boxes
                boxes, scores = obj_detect.detect(frame['img'])

                # Store ReID embeddings
                det_crops = crop_imgs(img, boxes.cpu().numpy(), tforms)
                det_reid = reid_model(det_crops.cuda())

                det = {'boxes': boxes.cpu(),
                      'scores': scores.cpu(),
                      'reid': det_reid.cpu()}


                # Store ground truth box info
                if 'gt' in frame and len(frame['gt']) > 0:
                    gt_ids = torch.as_tensor(list(frame['gt'].keys()))
                    gt_boxes = torch.cat([frame['gt'][id_.item()] for id_ in gt_ids], dim=0)
                    gt_vis = torch.cat([frame['vis'][id_.item()] for id_ in gt_ids], dim=0)
                    gt_crops = crop_imgs(img, gt_boxes.cpu().numpy(), tforms)
                    gt_reid = reid_model(gt_crops.cuda())

                    gt = {'ids': gt_ids.cpu(),
                          'boxes': gt_boxes.cpu(),
                          'vis': gt_vis.cpu(),
                          'reid': gt_reid.cpu()}
                else:
                    gt = None

                db[str(seq)].append({'det': det, 'gt': gt})
        assert len(db[str(seq)]) == len(data_loader)

    torch.save(db, osp.join(gnn_root_dir, 'data', stored_data_filename.format(train_test)))

Processing sequence MOT16-04


0it [00:00, ?it/s]

Processing sequence MOT16-05


0it [00:00, ?it/s]

Processing sequence MOT16-09


...

0it [00:00, ?it/s]

Processing sequence MOT16-08


0it [00:00, ?it/s]

Processing sequence MOT16-12


0it [00:00, ?it/s]

In [ ]:
train_db = torch.load(osp.join(gnn_root_dir, 'data/preprocessed_data_train_.pth'))

## Exercise Part 0 - Assignment's 1 ReIDHungarianTracker

We start by providing a sample solution of the ``ReIDTracker`` from Exercise 1.
It will serve as our baseline.


Recall that this tracker works by performing frame-to-frame bipartite matching between newly detected boxes and past tracks based on ReID distance. Whenever a past track cannot be matched, its killed. And whenever, a newly detected box cannot be match, it starts a new trajectory.

**NOTE**: We have modified the ``compute_distance`` function in ``data_association`` from last week to include a thresshold on ReID distance (if ReID distance >0.1, matching is not possible). This is important to prevent our tracker from reusing tracks for very dissimilar objects.

Results:
```
          IDF1   IDP   IDR  Rcll  Prcn  GT  MT  PT ML   FP    FN IDs   FM  MOTA  MOTP
MOT16-02 40.5% 57.5% 31.3% 52.2% 96.1%  62  12  38 12  390  8873 203  210 49.1% 0.090
MOT16-05 54.4% 64.4% 47.1% 68.8% 94.0% 133  54  67 12  305  2156 330  149 59.7% 0.142
MOT16-09 49.9% 61.7% 41.9% 66.3% 97.7%  26  13  12  1   82  1793  77   76 63.3% 0.082
MOT16-11 61.1% 67.4% 55.9% 80.2% 96.6%  75  44  24  7  266  1871 176   91 75.5% 0.083
OVERALL  49.6% 62.3% 41.2% 63.5% 96.1% 296 123 141 32 1043 14693 786  526 59.0% 0.097
```


In [ ]:
_UNMATCHED_COST=255
class ReIDHungarianTracker(ReIDTracker):
    def data_association(self, boxes, scores, pred_features):
        """Refactored from previous implementation to split it onto distance computation and track management"""
        if self.tracks:
            track_boxes = torch.stack([t.box for t in self.tracks], axis=0)
            track_features = torch.stack([t.get_feature() for t in self.tracks], axis=0)

            distance = self.compute_distance_matrix(track_features, pred_features,
                                                    track_boxes, boxes, metric_fn=cosine_distance)

            # Perform Hungarian matching.
            row_idx, col_idx = linear_assignment(distance)
            self.update_tracks(row_idx, col_idx,distance, boxes, scores, pred_features)


        else:
            # No tracks exist.
            self.add(boxes, scores, pred_features)

    def update_tracks(self, row_idx, col_idx, distance, boxes, scores, pred_features):
        """Updates existing tracks and removes unmatched tracks.
           Reminder: If the costs are equal to _UNMATCHED_COST, it's not a
           match.
        """
        track_ids = [t.id for t in self.tracks]

        unmatched_track_ids = []
        seen_track_ids = []
        seen_box_idx = []
        for track_idx, box_idx in zip(row_idx, col_idx):
            costs = distance[track_idx, box_idx]
            internal_track_id = track_ids[track_idx]
            seen_track_ids.append(internal_track_id)
            if costs == _UNMATCHED_COST:
                unmatched_track_ids.append(internal_track_id)
            else:
                self.tracks[track_idx].box = boxes[box_idx]
                self.tracks[track_idx].add_feature(pred_features[box_idx])
                seen_box_idx.append(box_idx)

        unseen_track_ids = set(track_ids) - set(seen_track_ids)
        unmatched_track_ids.extend(list(unseen_track_ids))
        self.tracks = [t for t in self.tracks
                       if t.id not in unmatched_track_ids]


        # Add new tracks.
        new_boxes_idx = set(range(len(boxes))) - set(seen_box_idx)
        new_boxes = [boxes[i] for i in new_boxes_idx]
        new_scores = [scores[i] for i in new_boxes_idx]
        new_features = [pred_features[i] for i in new_boxes_idx]
        self.add(new_boxes, new_scores, new_features)


In [ ]:
val_sequences = MOT16Sequences('MOT16-reid', root_dir = osp.join(root_dir, 'data/MOT16'), vis_threshold=0.)

In [ ]:
tracker = ReIDHungarianTracker(None)
run_tracker(val_sequences, db=train_db, tracker=tracker, output_dir=None)

Tracking: MOT16-02
Tracks found: 362
Runtime for MOT16-02: 5.7 s.
Tracking: MOT16-05
Tracks found: 460
Runtime for MOT16-05: 2.0 s.
Tracking: MOT16-09
Tracks found: 97
Runtime for MOT16-09: 1.3 s.
Tracking: MOT16-11
Tracks found: 290
Runtime for MOT16-11: 2.9 s.
Runtime for all sequences: 11.9 s.
          IDF1   IDP   IDR  Rcll  Prcn  GT  MT  PT ML   FP    FN IDs   FM  MOTA  MOTP
MOT16-02 40.8% 58.0% 31.5% 52.2% 96.2%  62  12  37 13  383  8875 204  210 49.1% 0.090
MOT16-05 54.4% 64.4% 47.1% 68.8% 94.1% 133  55  66 12  299  2160 330  150 59.7% 0.142
MOT16-09 49.9% 61.7% 41.9% 66.4% 97.8%  26  13  12  1   81  1791  76   76 63.4% 0.082
MOT16-11 61.4% 67.7% 56.2% 80.2% 96.6%  75  43  25  7  266  1869 173   89 75.5% 0.083
OVERALL  49.8% 62.6% 41.3% 63.5% 96.1% 296 123 140 33 1029 14695 783  525 59.0% 0.097


,idf1,idp,idr,recall,precision,num_unique_objects,mostly_tracked,partially_tracked,mostly_lost,num_false_positives,num_misses,num_switches,num_fragmentations,mota,motp
MOT16-02,0.407883,0.579542,0.314676,0.522362,0.962038,62,12,37,13,383,8875,204,210,0.490770,0.090317
MOT16-05,0.543723,0.643790,0.470580,0.687726,0.940862,133,55,66,12,299,2160,330,150,0.596791,0.142226
MOT16-09,0.499329,0.617427,0.419155,0.663662,0.977593,26,13,12,1,81,1791,76,76,0.634178,0.082109
MOT16-11,0.613817,0.676625,0.561679,0.801929,0.966041,75,43,25,7,266,1869,173,89,0.755405,0.082556
OVERALL,0.497637,0.625503,0.413175,0.634988,0.961306,296,123,140,33,1029,14695,783,525,0.589980,0.096544


## Exercise Part I - Long-Term ReID Tracker


The tracker above has an obvious limitation: whenever a track cannot be matched with the detections of a given frame the track will be killed. This means that if our detector misses an object in a single frame (due to e.g. occlusion), we will not be able to recover that track, and we will start a new one.

To fix this issue, we would like to allow our tracker to maintain tracks that are not matched during data association. We will refer to these tracks as **inactive**. During data association, we will try to match the detected boxes for the current frame to both tracks that are active (i.e. tracks that we were able to match in the previous frame) as well as those that are inactive. Therefore, if a detector misses an object in a frame and the object reappears after a few frames, we will still be able to match it to its corresponding track, instead of creating a new one.

In order to adapt our tracker to have this behavior, we will use the `inactive` attribute from the `track` class (see `tracker/tracker.py`. This attribute will be assigned an integer indicating for how many frames a track has remained unmatched. Whenever we are able to match the track `t`, we will set `t.inactive=0` and, naturally, when tracks are initialized, the class constructor sets `inactive=0`.

Your job is to maintain the `inactive` attribute of all tracks being kept by tracker so that its value represents the number of frames for which the track has been unmatched. Additionally, we introduce a `patience` parameter. Whenever a track has been inactive for more than `inactive` frames. it will need to be killed.

Results should approximately be around:

```
          IDF1   IDP   IDR  Rcll  Prcn  GT  MT  PT ML   FP    FN IDs   FM  MOTA  MOTP
MOT16-02 45.9% 65.1% 35.4% 52.2% 96.1%  62  12  37 13  390  8873 130  210 49.4% 0.090
MOT16-05 63.4% 75.0% 54.9% 68.8% 94.0% 133  54  67 12  305  2156 283  149 60.3% 0.142
MOT16-09 52.5% 64.9% 44.1% 66.3% 97.7%  26  13  12  1   82  1793  49   76 63.9% 0.083
MOT16-11 68.3% 75.3% 62.5% 80.2% 96.6%  75  44  24  7  266  1871 136   90 75.9% 0.083
OVERALL  55.7% 70.0% 46.2% 63.5% 96.1% 296 123 140 33 1043 14693 598  525 59.4% 0.097
```


In [ ]:
class LongTermReIDHungarianTracker(ReIDHungarianTracker):
    def __init__(self, patience, *args, **kwargs):
        """ Add a patience parameter"""
        self.patience=patience
        super().__init__(*args, **kwargs)

    def update_results(self):
        """Only store boxes for tracks that are active"""
        for t in self.tracks:
            if t.id not in self.results.keys():
                self.results[t.id] = {}
            if t.inactive == 0: # Only change
                self.results[t.id][self.im_index] = np.concatenate([t.box.cpu().numpy(), np.array([t.score])])

        self.im_index += 1

    def update_tracks(self, row_idx, col_idx, distance, boxes, scores, pred_features):
        track_ids = [t.id for t in self.tracks]

        unmatched_track_ids = []
        seen_track_ids = []
        seen_box_idx = []
        for track_idx, box_idx in zip(row_idx, col_idx):
            costs = distance[track_idx, box_idx]
            internal_track_id = track_ids[track_idx]
            seen_track_ids.append(internal_track_id)
            if costs == _UNMATCHED_COST:
                unmatched_track_ids.append(internal_track_id)

            else:
                self.tracks[track_idx].box = boxes[box_idx]
                self.tracks[track_idx].add_feature(pred_features[box_idx])

                # Note: the track is matched, therefore, inactive is set to 0
                self.tracks[track_idx].inactive=0
                seen_box_idx.append(box_idx)


        unseen_track_ids = set(track_ids) - set(seen_track_ids)
        unmatched_track_ids.extend(list(unseen_track_ids))
        ##################
        ### TODO starts
        ##################

        # Update the `inactive` attribute for those tracks that have been
        # not been matched. kill those for which the inactive parameter
        # is > self.patience

        selected_tracks = []
        for track_idx, i in enumerate(self.tracks):
            if i.id in unmatched_track_ids:
                self.tracks[track_idx].inactive += 1

            if i.inactive > self.patience:
                selected_tracks.append(i.id)

        # <-- Needs to be updated
        self.tracks = [i for i in self.tracks
                       if i.id not in selected_tracks]

        ##################
        ### TODO ends
        ##################

        new_boxes_idx = set(range(len(boxes))) - set(seen_box_idx)
        new_boxes = [boxes[i] for i in new_boxes_idx]
        new_scores = [scores[i] for i in new_boxes_idx]
        new_features = [pred_features[i] for i in new_boxes_idx]
        self.add(new_boxes, new_scores, new_features)

In [ ]:
tracker = LongTermReIDHungarianTracker(patience=20, obj_detect=None)
run_tracker(val_sequences, db=train_db, tracker=tracker, output_dir=None)

Tracking: MOT16-02
Tracks found: 203
Runtime for MOT16-02: 6.2 s.
Tracking: MOT16-05
Tracks found: 352
Runtime for MOT16-05: 2.1 s.
Tracking: MOT16-09
Tracks found: 65
Runtime for MOT16-09: 1.4 s.
Tracking: MOT16-11
Tracks found: 220
Runtime for MOT16-11: 3.2 s.
Runtime for all sequences: 13.0 s.
          IDF1   IDP   IDR  Rcll  Prcn  GT  MT  PT ML   FP    FN IDs   FM  MOTA  MOTP
MOT16-02 45.3% 64.4% 35.0% 52.2% 96.2%  62  12  37 13  383  8875 131  210 49.5% 0.090
MOT16-05 63.5% 75.2% 54.9% 68.8% 94.1% 133  55  66 12  299  2160 281  150 60.4% 0.142
MOT16-09 50.6% 62.5% 42.4% 66.4% 97.8%  26  13  12  1   81  1791  49   76 63.9% 0.082
MOT16-11 67.6% 74.6% 61.9% 80.2% 96.6%  75  43  25  7  266  1869 137   88 75.9% 0.083
OVERALL  55.0% 69.2% 45.7% 63.5% 96.1% 296 123 140 33 1029 14695 598  524 59.5% 0.097


,idf1,idp,idr,recall,precision,num_unique_objects,mostly_tracked,partially_tracked,mostly_lost,num_false_positives,num_misses,num_switches,num_fragmentations,mota,motp
MOT16-02,0.453296,0.644068,0.349712,0.522362,0.962038,62,12,37,13,383,8875,131,210,0.494699,0.090315
MOT16-05,0.634762,0.751582,0.549371,0.687726,0.940862,133,55,66,12,299,2160,281,150,0.603875,0.142168
MOT16-09,0.505593,0.625173,0.424413,0.663662,0.977593,26,13,12,1,81,1791,49,76,0.639249,0.082153
MOT16-11,0.676472,0.745691,0.619012,0.801929,0.966041,75,43,25,7,266,1869,137,88,0.759220,0.082573
OVERALL,0.550440,0.691874,0.457016,0.634988,0.961306,296,123,140,33,1029,14695,598,524,0.594575,0.096544


## Exercise Part II - Building a tracker based on Neural Message Passing

Our ``LongTermReIDHungarianTracker`` is still limited when compared to current modern trackers.

Firstly, it relies solely on appearance to predict similarity scores between objectes. This can be problematic whenever appearance alone may not discriminative, and it'd be best to also take into account object position and size attributes. Secondly, our tracker can only account for pairwise similarities among objects. Ideally, we would like it to also consider higher-order information.

To address these limitations. We will now build a tracker that will combine both apperance and position information with a Message Passing Neural Network, inspired by the approach presented in [Learning a Neural Solver for Multiple Object Tracking, CVPR 2020](https://arxiv.org/abs/1912.07515)

The overall idea will be to build, for every tracking step, a bipartite graph containing two sets of nodes: past tracks, and detections in the current frame. We will initialize node features with ReID embeddings, and edge features with relative position features and ReID distance. We will use an MPN to refine these edge embeddings. The learning task will be to classify the edge embeddings in this graph, which is equivalent to predicting the entries of our data association similarity matrix.


### Building an MPN for Bipartite Graphs

We will first build a Neural Message Passing layer based on the Graph Networks framework introduced in [Relational inductive biases, deep learning, and graph networks, arXiv 2020](https://arxiv.org/abs/1806.01261), as explained in the *A More General Framework* slides of [Lecture 5](https://www.moodle.tum.de/pluginfile.php/2928927/mod_resource/content/1/5.MOT2.pdf) (slides 70 to 75).

We will be using a bipartite graph, i.e., we will have two sets of nodes $A$ (past tracks), and $B$ (detections), and our set of edges will be $A\times B$. That is, we will connect every pair of past tracks and detections.

We will have initial node features (i.e. reid embeddings) matrices: $X_A$ and $X_B$ and an initial edge features tensor $E$.

$X_A$ and $X_B$ have shape $|A|\times \text{node\_dim}$ and $|B|\times \text{node\_dim}$, respectively.

$E$ has shape $|A| \times |B| \times \text{edge\_dim}$. Its $(i, j)$ entry contains the edge features of node $i$ in $A$ and node $j$ in $B$.

With the given layer, we will produce new node feature matrices $X_A'$ and $X_B'$ and edge features $E'$ with the same dimensions.
Please refer to the formulas in the slides and figure how to apply them in this setting.

You are asked to implement both the node and edge update steps in the class below

**NOTE 1**: Working with a bipartite graph allows us to vectorize all operations in the formulas in a straightforward manner (keep in mind that we store edge features in a matrix). Given a node in $A$, it is connected to all nodes in $B$.

**NOTE 2**: You do not need to care about batching several graphs. This implementation will only work with a single graph at a time.

In [ ]:
from torch import nn

class BipartiteNeuralMessagePassingLayer(nn.Module):
    def __init__(self, node_dim, edge_dim, dropout=0.):
        super().__init__()

        edge_in_dim  = 2*node_dim + 2*edge_dim # 2*edge_dim since we always concatenate initial edge features
        self.edge_mlp = nn.Sequential(*[nn.Linear(edge_in_dim, edge_dim), nn.ReLU(), nn.Dropout(dropout),
                                    nn.Linear(edge_dim, edge_dim), nn.ReLU(), nn.Dropout(dropout)])

        node_in_dim  = node_dim + edge_dim
        self.node_mlp = nn.Sequential(*[nn.Linear(node_in_dim, node_dim), nn.ReLU(), nn.Dropout(dropout),
                                        nn.Linear(node_dim, node_dim), nn.ReLU(), nn.Dropout(dropout)])

    def edge_update(self, edge_embeds, nodes_a_embeds, nodes_b_embeds):
        """
        Node-to-edge updates, as descibed in slide 71, lecture 5.
        Args:
            edge_embeds: torch.Tensor with shape (|A|, |B|, 2 x edge_dim)
            nodes_a_embeds: torch.Tensor with shape (|A|, node_dim)
            nodes_a_embeds: torch.Tensor with shape (|B|, node_dim)

        returns:
            updated_edge_feats = torch.Tensor with shape (|A|, |B|, edge_dim)
        """

        n_nodes_a, n_nodes_b, _  = edge_embeds.shape

        ########################
        #### TODO starts
        ########################

        nodes_a = nodes_a_embeds.unsqueeze(1).expand(-1, n_nodes_b, -1)
        nodes_b = nodes_b_embeds.unsqueeze(0).expand(n_nodes_a, -1, -1)

        edge_in = torch.cat((edge_embeds, nodes_a, nodes_b), dim=-1) # has shape (|A|, |B|, 2*node_dim + 2*edge_dim)

        ########################
        #### TODO ends
        ########################


        return self.edge_mlp(edge_in)

    def node_update(self, edge_embeds, nodes_a_embeds, nodes_b_embeds):
        """
        Edge-to-node updates, as descibed in slide 75, lecture 5.

        Args:
            edge_embeds: torch.Tensor with shape (|A|, |B|, edge_dim)
            nodes_a_embeds: torch.Tensor with shape (|A|, node_dim)
            nodes_b_embeds: torch.Tensor with shape (|B|, node_dim)

        returns:
            tuple(
                updated_nodes_a_embeds: torch.Tensor with shape (|A|, node_dim),
                updated_nodes_b_embeds: torch.Tensor with shape (|B|, node_dim)
                )
        """

        ########################
        #### TODO starts
        ########################

        # NOTE: Use 'sum' as aggregation function

        edge_a = edge_embeds.sum(dim=1)
        nodes_a_in = torch.cat((nodes_a_embeds, edge_a), dim=-1)
        # Has shape (|A|, node_dim + edge_dim)

        edge_b = edge_embeds.sum(dim=0)
        nodes_b_in = torch.cat((nodes_b_embeds, edge_b), dim=-1)
        # Has shape (|B|, node_dim + edge_dim)

        ########################
        #### TODO ends
        ########################



        nodes_a = self.node_mlp(nodes_a_in)
        nodes_b = self.node_mlp(nodes_b_in)

        return nodes_a, nodes_b

    def forward(self, edge_embeds, nodes_a_embeds, nodes_b_embeds):
        edge_embeds_latent = self.edge_update(edge_embeds, nodes_a_embeds, nodes_b_embeds)
        nodes_a_latent, nodes_b_latent = self.node_update(edge_embeds_latent, nodes_a_embeds, nodes_b_embeds)

        return edge_embeds_latent, nodes_a_latent, nodes_b_latent

## Building the entire network to predict similarities
We now build the network that generates initial node and edge features, performs neural message passing, and classifies edges in order to produce the final costs that we will use for data association.

You need to implement the method that computes the initial edge features. You can can follow [1] and, given a two bounding boxes $(x_i, y_i, w_i, h_i)$ and  $(x_j, y_j, w_j, h_j)$ and timestamps $t_i$ and $t_j$, compute an initial 5-dimensional edge feature vector as:
$$ E_(i, j) = \left (\frac{2(x_j - x_i)}{h_i + h_j}, \frac{2(y_j - y_i)}{h_i + h_j}, \log{\frac{h_i}{h_j}}, \log{\frac{w_i}{w_j}}, t_j - t_i \right )$$


Feel free to engineer your own features (e.g. use IoU, etc.)

In [ ]:
from torch.nn import functional as F
class AssignmentSimilarityNet(nn.Module):
    def __init__(self, reid_network, node_dim, edge_dim, reid_dim, edges_in_dim, num_steps, dropout=0.):
        super().__init__()
        self.reid_network = reid_network
        self.graph_net = BipartiteNeuralMessagePassingLayer(node_dim=node_dim, edge_dim=edge_dim, dropout=dropout)
        self.num_steps = num_steps
        self.cnn_linear = nn.Linear(reid_dim, node_dim)
        self.edge_in_mlp = nn.Sequential(*[nn.Linear(edges_in_dim, edge_dim), nn.ReLU(), nn.Dropout(dropout), nn.Linear(edge_dim, edge_dim), nn.ReLU(),nn.Dropout(dropout)])
        self.classifier = nn.Sequential(*[nn.Linear(edge_dim, edge_dim), nn.ReLU(), nn.Linear(edge_dim, 1)])


    def compute_edge_feats(self, track_coords, current_coords, track_t, curr_t):
        """
        Computes initial edge feature tensor

        Args:
            track_coords: track's frame box coordinates, given by top-left and bottom-right coordinates
                          torch.Tensor with shape (num_tracks, 4)
            current_coords: current frame box coordinates, given by top-left and bottom-right coordinates
                            has shape (num_boxes, 4)

            track_t: track's timestamps, torch.Tensor with with shape (num_tracks, )
            curr_t: current frame's timestamps, torch.Tensor withwith shape (num_boxes,)


        Returns:
            tensor with shape (num_trakcs, num_boxes, 5) containing pairwise
            position and time difference features
        """

        ########################
        #### TODO starts
        ########################

        # NOTE 1: we recommend you to use box centers to compute distances
        # in the x and y coordinates.

        # NOTE 2: Check out the  code inside train_one_epoch function and
        # LongTrackTrainingDataset class a few cells below to debug this

        track_centers = track_coords.view(-1, 2, 2).mean(-2)
        curr_centers = current_coords.view(-1, 2, 2).mean(-2)

        track_w = track_coords[..., 2] - track_coords[..., 0]
        track_h = track_coords[..., 3] - track_coords[..., 1]

        curr_w = current_coords[..., 2] - current_coords[..., 0]
        curr_h = current_coords[..., 3] - current_coords[..., 1]

        assert (track_w >0).all() and (track_h >0).all()

        denom = (track_w.unsqueeze(1) + curr_w.unsqueeze(0)) / 2
        dist_x = (track_centers[..., 0].unsqueeze(1) - curr_centers[..., 0].unsqueeze(0))
        dist_y = (track_centers[..., 1].unsqueeze(1) - curr_centers[..., 1].unsqueeze(0))

        dist_x = dist_x / denom
        dist_y = dist_y / denom

        dist_w = torch.log(track_w.unsqueeze(1) / curr_w.unsqueeze(0))
        dist_h = torch.log(track_h.unsqueeze(1) / curr_h.unsqueeze(0))

        dist_t = (track_t.unsqueeze(1) - curr_t.unsqueeze(0)).type(dist_h.dtype).to(dist_h.device)

        edge_feats = torch.stack([dist_x, dist_y, dist_w, dist_h, dist_t])
        edge_feats = edge_feats.permute(1, 2, 0)

        ########################
        #### TODO ends
        ########################

        return edge_feats # has shape (num_trakcs, num_boxes, 5)


    def forward(self, track_app, current_app, track_coords, current_coords, track_t, curr_t):
        """
        Args:
            track_app: track's reid embeddings, torch.Tensor with shape (num_tracks, 512)
            current_app: current frame detections' reid embeddings, torch.Tensor with shape (num_boxes, 512)
            track_coords: track's frame box coordinates, given by top-left and bottom-right coordinates
                          torch.Tensor with shape (num_tracks, 4)
            current_coords: current frame box coordinates, given by top-left and bottom-right coordinates
                            has shape (num_boxes, 4)

            track_t: track's timestamps, torch.Tensor with with shape (num_tracks, )
            curr_t: current frame's timestamps, torch.Tensor withwith shape (num_boxes,)

        Returns:
            classified edges: torch.Tensor with shape (num_steps, num_tracks, num_boxes),
                             containing at entry (step, i, j) the unnormalized probability that track i and
                             detection j are a match, according to the classifier at the given neural message passing step
        """

        # Get initial edge embeddings to
        dist_reid = cosine_distance(track_app, current_app)
        pos_edge_feats = self.compute_edge_feats(track_coords, current_coords, track_t, curr_t)
        edge_feats = torch.cat((pos_edge_feats, dist_reid.unsqueeze(-1)), dim=-1)
        edge_embeds = self.edge_in_mlp(edge_feats)
        initial_edge_embeds = edge_embeds.clone()

        # Get initial node embeddings, reduce dimensionality from 512 to node_dim
        track_embeds = F.relu(self.cnn_linear(track_app))
        curr_embeds =F.relu(self.cnn_linear(current_app))

        classified_edges = []
        for _ in range(self.num_steps):
            edge_embeds = torch.cat((edge_embeds, initial_edge_embeds), dim=-1)
            edge_embeds, track_embeds, curr_embeds = self.graph_net(edge_embeds=edge_embeds,
                                                                    nodes_a_embeds=track_embeds,
                                                                    nodes_b_embeds=curr_embeds)

            classified_edges.append(self.classifier(edge_embeds))

        return torch.stack(classified_edges).squeeze(-1)

## Putting everything together

Finally, we incorporate our ``AssignmentSimilarityNet`` into our tracker. We can keep everything as in ``LongTermReIDHungarianTracker`` except for the distance computation, which is now directly obtained via a forward pass through AssignmentSimilarityNet.

In [ ]:
_UNMATCHED_COST=255
class MPNTracker(LongTermReIDHungarianTracker):
    def __init__(self, assign_net, *args, **kwargs):
        self.assign_net = assign_net
        super().__init__(*args, **kwargs)

    def data_association(self, boxes, scores, pred_features):
        if self.tracks:
            track_boxes = torch.stack([t.box for t in self.tracks], axis=0).cuda()
            track_features = torch.stack([t.get_feature() for t in self.tracks], axis=0).cuda()

            # Hacky way to recover the timestamps of boxes and tracks
            curr_t = self.im_index * torch.ones((pred_features.shape[0],)).cuda()
            track_t = torch.as_tensor([self.im_index - t.inactive - 1 for t in self.tracks]).cuda()

            ########################
            #### TODO starts
            ########################

            # Do a forward pass through self.assign_net to obtain our costs.
            # Note: self.assign_net will return unnormalized probabilities.
            # Make sure to apply the sigmoid function to them!
            pred_sim = torch.sigmoid(self.assign_net.forward(track_app = track_features.cuda(),
                                                             current_app = pred_features.cuda(),
                                                             track_coords = track_boxes.cuda(),
                                                             current_coords=boxes.cuda(),
                                                             track_t = track_t,
                                                             curr_t = curr_t)).detach().cpu().numpy()


            ########################
            #### TODO ends
            ########################

            pred_sim = pred_sim[-1]  # Use predictions at last message passing step
            distance = (1- pred_sim)

            # Do not allow mataches when sim < 0.5, to avoid low-confident associations
            distance = np.where(pred_sim < 0.5, _UNMATCHED_COST, distance)

            # Perform Hungarian matching.
            row_idx, col_idx = linear_assignment(distance)
            self.update_tracks(row_idx, col_idx,distance, boxes, scores, pred_features)


        else:
            # No tracks exist.
            self.add(boxes, scores, pred_features)

## Training and evaluating our model

We provide all boilerplate code for training our neural message passing based
tracker, as well as evaluating.

Under the hood, we are sampling frames randomly from our training sequences, and then sampling boxes from past frames as past_tracks to generate our
training
data. Check out `LongTrackTrainingDataset` for details.

We train the model with a weighted cross-entropy loss
to account for the class imbalance. Check out `train_one_epoch` if you're
interested.

No need to write any code from your side here!


In [ ]:
from gnn.dataset import LongTrackTrainingDataset
from torch.utils.data import DataLoader
from gnn.trainer import train_one_epoch

MAX_PATIENCE = 20
MAX_EPOCHS = 20
EVAL_FREQ = 1


# Define our model, and init
assign_net = AssignmentSimilarityNet(reid_network=None, # Not needed since we work with precomputed features
                                     node_dim=32,
                                     edge_dim=64,
                                     reid_dim=512,
                                     edges_in_dim=6,
                                     num_steps=10).cuda()

# We only keep two sequences for validation. You can
dataset = LongTrackTrainingDataset(dataset='MOT16-train_wo_val2',
                                   db=train_db,
                                   root_dir= osp.join(root_dir, 'data/MOT16'),
                                   max_past_frames = MAX_PATIENCE,
                                   vis_threshold=0.25)

data_loader = DataLoader(dataset, batch_size=8, collate_fn = lambda x: x,
                         shuffle=True, num_workers=2, drop_last=True)
device = torch.device('cuda')
optimizer = torch.optim.Adam(assign_net.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5)

We only leave 2 sequences for validation in order to maximize
the amount of training data. For your convenience, here are the
 LongTermReIDTracker results on them. Your validation IDF1 scores should show an improvement over them.
```

          IDF1   IDP   IDR  Rcll  Prcn  GT MT PT ML  FP    FN IDs   FM  MOTA  MOTP
MOT16-02 45.9% 65.1% 35.4% 52.2% 96.1%  62 12 37 13 390  8873 130  210 49.4% 0.090
MOT16-11 68.3% 75.3% 62.5% 80.2% 96.6%  75 44 24  7 266  1871 136   90 75.9% 0.083
OVERALL  54.3% 69.6% 44.5% 61.7% 96.3% 137 56 61 20 656 10744 266  300 58.4% 0.087
```



Let's start training!

Note that we have observed quite a lot of noise in validation scores among epochs and runs. This can be explained due to the small size of our training and
validation sets, that's why we perform early stopping to obtain the best performing model on validation. In addition, changing the experiment seed and/or relaunching the training might help in case you are suspecting that noise might be influencing your scores.

In [ ]:
best_idf1 = 0.
for epoch in range(1, MAX_EPOCHS + 1):
    print(f"-------- EPOCH {epoch:2d} --------")
    train_one_epoch(model = assign_net, data_loader=data_loader, optimizer=optimizer, print_freq=100)
    scheduler.step()

    if epoch % EVAL_FREQ == 0:
        tracker =  MPNTracker(assign_net=assign_net.eval(), obj_detect=None, patience=MAX_PATIENCE)
        val_sequences = MOT16Sequences('MOT16-val2', osp.join(root_dir, 'data/MOT16'), vis_threshold=0.)
        res = run_tracker(val_sequences, db=train_db, tracker=tracker, output_dir=None)
        idf1 = res.loc['OVERALL']['idf1']
        if idf1 > best_idf1:
            best_idf1 = idf1
            torch.save(assign_net.state_dict(), osp.join(root_dir, 'output', 'best_ckpt.pth'))


-------- EPOCH  1 --------


100it [00:25,  4.09it/s]

Iter 100. Loss: 0.593. Accuracy: 0.953. Recall: 0.769. Precision: 0.560


200it [00:51,  4.01it/s]

Iter 200. Loss: 0.100. Accuracy: 0.989. Recall: 0.987. Precision: 0.871


...

Iter 300. Loss: 0.011. Accuracy: 0.999. Recall: 0.998. Precision: 0.988


400it [01:41,  3.96it/s]

Iter 400. Loss: 0.010. Accuracy: 0.999. Recall: 0.999. Precision: 0.991


476it [02:00,  3.96it/s]


Tracking: MOT16-02
Tracks found: 108
Runtime for MOT16-02: 8.9 s.
Tracking: MOT16-11
Tracks found: 81
Runtime for MOT16-11: 8.8 s.
Runtime for all sequences: 17.6 s.
          IDF1   IDP   IDR  Rcll  Prcn  GT MT PT ML  FP    FN IDs   FM  MOTA  MOTP
MOT16-02 47.3% 67.3% 36.5% 52.2% 96.2%  62 11 38 13 383  8875 102  217 49.6% 0.095
MOT16-11 69.9% 77.1% 64.0% 80.2% 96.6%  75 43 25  7 266  1869  34   88 77.0% 0.083
OVERALL  55.8% 71.6% 45.8% 61.7% 96.4% 137 54 63 20 649 10744 136  305 58.8% 0.090


# Exercise submission

After executing the followinc cell the `Colab Notebooks/cv3dst_exercise/output` directory in your Google Drive should contain multiple `MOT16-XY.txt` files.

For the final submission you have to process the test sequences and upload the zipped prediction files to our server. See moodle for a guide how to upload the results.

Note that this time, you only have to evaluate three sequences!

In [ ]:
best_ckpt = torch.load(osp.join(root_dir, 'output', 'best_ckpt.pth'))
assign_net.load_state_dict(best_ckpt)

tracker =  MPNTracker(assign_net=assign_net.eval(), obj_detect=None, patience=MAX_PATIENCE)
test_db = torch.load(osp.join(gnn_root_dir, 'data/preprocessed_data_test_2.pth'))
val_sequences = MOT16Sequences('MOT16-test', osp.join(root_dir, 'data/MOT16'), vis_threshold=0.)
run_tracker(val_sequences, db=test_db, tracker=tracker, output_dir=osp.join(root_dir, 'output'))

Tracking: MOT16-01
No GT evaluation data available.
Tracks found: 94
Runtime for MOT16-01: 3.8 s.
Writing predictions to: gdrive/My Drive/Colab Notebooks/cv3dst_exercise/output/MOT16-01.txt
Tracking: MOT16-08
No GT evaluation data available.
Tracks found: 161
Runtime for MOT16-08: 5.0 s.
Writing predictions to: gdrive/My Drive/Colab Notebooks/cv3dst_exercise/output/MOT16-08.txt
Tracking: MOT16-12
No GT evaluation data available.
Tracks found: 159
Runtime for MOT16-12: 5.9 s.
Writing predictions to: gdrive/My Drive/Colab Notebooks/cv3dst_exercise/output/MOT16-12.txt
Runtime for all sequences: 14.7 s.
